##### The files are:

1. personal_info.csv - personal information such as name, gender, etc. (one row per person)
2. vehicles.csv - what vehicle people own (one row per person)
3. employment.csv - where a person is employed (one row per person)
4. update_status.csv - when the person's data was created and last updated

Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

### Goal 1
Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

##### Let see how data looks in each csv file

In [1]:
with open('personal_info.csv') as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        print(vals)
        if i == 2:
            break

ssn,first_name,last_name,gender,language

100-53-9824,Sebastiano,Tester,Male,Icelandic

101-71-4702,Cayla,MacDonagh,Female,Lao



In [2]:
personal_info_type = ['INT', 'STRING', 'STRING', 'STRING','STRING']

In [3]:
with open('vehicles.csv') as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        print(vals)
        if i == 2:
            break

ssn,vehicle_make,vehicle_model,model_year

100-53-9824,Oldsmobile,Bravada,1993

101-71-4702,Ford,Mustang,1997



In [4]:
vehicles_type= ['INT','STRING','STRING','INT']

In [5]:
with open('employment.csv') as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        print(vals)
        if i == 2:
            break

employer,department,employee_id,ssn

Stiedemann-Bailey,Research and Development,29-0890771,100-53-9824

Nicolas and Sons,Sales,41-6841359,101-71-4702



In [6]:
employment_type = ['STRING','STRING','INT','INT']

In [7]:
with open('update_status.csv') as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        print(vals)
        if i == 2:
            break

ssn,last_updated,created

100-53-9824,2017-10-07T00:14:42Z,2016-01-24T21:19:30Z

101-71-4702,2017-01-23T11:23:17Z,2016-01-27T04:32:57Z



In [8]:
status_type = ['INT','DATE','DATE']

In [9]:
from datetime import datetime
def type_casting(col_data,input_val):
    if col_data == 'DATE':
        return datetime.strptime(input_val, '%Y-%m-%dT%H:%M:%SZ').date()
    elif col_data == 'INT':
        return int(input_val.replace('-','_'))
    else:
        return input_val

In [10]:
def cast_each_row(col_data_type,row_val):
    return [type_casting(col_data, value) for col_data, value in zip(col_data_type, row_val)]

In [11]:
from collections import namedtuple
import csv
from itertools import islice
def gen_fun(input_file,col_data_type):
    with open(input_file) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        headers = next(iter(rows))
        named_tuple = namedtuple(input_file.replace('.','_'), headers)
        for row in rows:
            yield named_tuple(*cast_each_row(col_data_type, row))

In [33]:
file_path1='personal_info.csv'

personal_data = gen_fun(file_path1,personal_info_type)
for data in islice(personal_data, 3):
    print(data)

personal_info_csv(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
personal_info_csv(ssn=101714702, first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
personal_info_csv(ssn=101840356, first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')


In [34]:
file_path2='vehicles.csv'

vehicles_data = gen_fun(file_path2,vehicles_type)
for data in islice(vehicles_data, 3):
    print(data)

vehicles_csv(ssn=100539824, vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
vehicles_csv(ssn=101714702, vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
vehicles_csv(ssn=101840356, vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)


In [35]:
file_path3='employment.csv'

employment_data = gen_fun(file_path3,employment_type)
for data in islice(employment_data, 3):
    print(data)

employment_csv(employer='Stiedemann-Bailey', department='Research and Development', employee_id=290890771, ssn=100539824)
employment_csv(employer='Nicolas and Sons', department='Sales', employee_id=416841359, ssn=101714702)
employment_csv(employer='Connelly Group', department='Research and Development', employee_id=987952860, ssn=101840356)


In [36]:
file_path4='update_status.csv'

status_data = gen_fun(file_path4,status_type)
for data in islice(status_data, 3):
    print(data)

update_status_csv(ssn=100539824, last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24))
update_status_csv(ssn=101714702, last_updated=datetime.date(2017, 1, 23), created=datetime.date(2016, 1, 27))
update_status_csv(ssn=101840356, last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21))



#### Goal 2
Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns. Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

In [37]:
def ssn_match():
    for pers, emp, vehi, upd in zip(personal_data, employment_data, vehicles_data, status_data):
        all_fields = pers._fields + emp._fields + vehi._fields + upd._fields
        ssn_match_info = namedtuple("ssn_match_info", sorted(set(all_fields), key=all_fields.index))
        final_data = [*pers, *emp, *vehi, *upd]
        yield ssn_match_info(*sorted(set(final_data), key=final_data.index))

In [39]:
ssn_matched=ssn_match()
for data in islice(ssn_matched, 3):
    print(data)

ssn_match_info(ssn=105857486, first_name='Angelina', last_name='McAvey', gender='Female', language='Punjabi', employer='Roberts, Torphy and Dach', department='Human Resources', employee_id=774895332, vehicle_make='Chrysler', vehicle_model='300', model_year=2008, last_updated=datetime.date(2018, 2, 14), created=datetime.date(2016, 12, 15))
ssn_match_info(ssn=105915022, first_name='Moselle', last_name='Apfel', gender='Female', language='Latvian', employer='Lind-Jast', department='Marketing', employee_id=796418731, vehicle_make='Isuzu', vehicle_model='Hombre Space', model_year=2000, last_updated=datetime.date(2018, 3, 24), created=datetime.date(2016, 3, 24))
ssn_match_info(ssn=105917777, first_name='Audi', last_name='Roach', gender='Female', language='Estonian', employer='Bashirian-Lueilwitz', department='Engineering', employee_id=443328799, vehicle_make='Chevrolet', vehicle_model='Silverado 3500', model_year=2004, last_updated=datetime.date(2017, 5, 11), created=datetime.date(2016, 5, 31

#### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the last_updated field from the status_update file.

In [40]:
def stale_records(input_date):
    date = datetime.strptime(input_date, '%d-%m-%Y').date()
    for ssn_matched_data in ssn_match():
        if ssn_matched_data.last_updated < date:
            yield ssn_matched_data

In [41]:
input_date = '11-01-2018'
stale_record=stale_records(input_date)

In [42]:
from itertools import islice
for data in islice(stale_record, 4):
    print(data)

ssn_match_info(ssn=106351938, first_name='Mackenzie', last_name='Nussey', gender='Male', language='Swedish', employer='Windler, Marks and Haley', department='Services', employee_id=546271885, vehicle_make='GMC', vehicle_model='Sonoma Club', model_year=1992, last_updated=datetime.date(2017, 10, 21), created=datetime.date(2016, 9, 8))
ssn_match_info(ssn=106363293, first_name='Martino', last_name='Tregoning', gender='Male', language='Tok Pisin', employer='Leffler-Hahn', department='Accounting', employee_id=315735282, vehicle_make='Volkswagen', vehicle_model='Touareg', model_year=2008, last_updated=datetime.date(2017, 3, 18), created=datetime.date(2016, 5, 16))
ssn_match_info(ssn=110843641, first_name='Amberly', last_name='Huws', gender='Female', language='Papiamento', employer='Lueilwitz LLC', department='Marketing', employee_id=339146042, vehicle_make='Ford', vehicle_model='Mustang', model_year=1990, last_updated=datetime.date(2017, 9, 4), created=datetime.date(2016, 9, 12))
ssn_match_in

#### Goal 4
Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [43]:
all_record = ssn_match()
make_count_male = dict()
make_count_female = dict()
for _ in all_record:
    if _.gender == 'Male':
        make_count_male[_.vehicle_make] = make_count_male[_.vehicle_make] + 1 if _.vehicle_make in make_count_male else 1
    else:
        make_count_female[_.vehicle_make] = make_count_female[_.vehicle_make] + 1 if _.vehicle_make in make_count_female else 1

In [44]:
max_value = max(list(make_count_male.values()))
print([(k, v) for k, v in make_count_male.items() if v == max_value])

[('Ford', 41)]


In [45]:
max_value = max(list(make_count_female.values()))
print([(k, v) for k, v in make_count_female.items() if v == max_value])

[('Chevrolet', 47)]
